In [1]:
from utils import load_encoder_hparams_and_params
encoder, hparams, params = load_encoder_hparams_and_params("124M", "models")

2024-09-16 17:19:19.220873: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-16 17:19:19.437577: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-16 17:19:19.437613: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-09-16 17:19:20.518000: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-1

In [2]:
inputs = encoder.encode("all heroes wear capes")
 
inputs

[439, 10281, 5806, 1451, 274]

## Import weights one by one 

In [13]:
import numpy as np

# Load the .npy file
b = np.load('saved_weights/model_ln_f_b.npy', allow_pickle=True)
g = np.load('saved_weights/model_ln_f_g.npy', allow_pickle=True)
 
c_attn_b = np.load('saved_weights/model_h0_attn_c_attn_b.npy', allow_pickle=True)
c_attn_w = np.load('saved_weights/model_h0_attn_c_attn_w.npy', allow_pickle=True)
 

c_proj_b = np.load('saved_weights/model_h0_attn_c_proj_b.npy', allow_pickle=True) 
c_proj_w = np.load('saved_weights/model_h0_attn_c_proj_w.npy', allow_pickle=True)


## Helper funciton 

In [14]:
def attention(q, k, v, mask):
    return softmax(q @ k.T / np.sqrt(q.shape[-1]) + mask) @ v

In [15]:
def linear(x, w, b):  # [m, in], [in, out], [out] -> [m, out]
    return x @ w + b

In [19]:
def mha(x, c_attn, c_proj, n_head=1):  # [n_seq, n_embd] -> [n_seq, n_embd]
     x = linear(x, c_attn_w , c_attn_b)  # [n_seq, n_embd] -> [n_seq, 3*n_embd]

     qkv = np.split(x, 3, axis=-1)  # [n_seq, 3*n_embd] -> [3, n_seq, n_embd]

     qkv_heads = list(map(lambda x: np.split(x, n_head, axis=-1), qkv))  # [3, n_seq, n_embd] -> [3, n_head, n_seq, n_embd/n_head]

     causal_mask = (1 - np.tri(x.shape[0], dtype=x.dtype)) * -1e10  # [n_seq, n_seq]

     out_heads = [attention(q, k, v, causal_mask) for q, k, v in zip(*qkv_heads)]  # [3, n_head, n_seq, n_embd/n_head] -> [n_head, n_seq, n_embd/n_head]

     x = np.hstack(out_heads)  # [n_head, n_seq, n_embd/n_head] -> [n_seq, n_embd]

     x = linear(x, c_proj_w,c_proj_b )  # [n_seq, n_embd] -> [n_seq, n_embd]

     return x

In [20]:
def transformer_block(x, mlp, attn, ln_1, ln_2, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
     
    x =  mha(x, **attn, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

   

    return x

In [21]:
def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):
    x = wte[inputs] + wpe[range(len(inputs))]
    for block in blocks[:1]:
      x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x

In [22]:
def layer_norm(x, g, b, eps: float = 1e-5):
    print("Layer Norm Input Shapes:")
    print(f"x: {x}")
    print(f"x shape: {x.shape}")
    print(f"g shape: {g.shape}")
    print(f"b shape: {b.shape}")
    
    mean = np.mean(x, axis=-1, keepdims=True)
    print(f"mean: {mean}")
    variance = np.var(x, axis=-1, keepdims=True)
    print(f"variance: {variance}")
    
    x_normalized = (x - mean) / np.sqrt(variance + eps)
    print(f"x_normalized: {x_normalized}")
    
    result = g * x_normalized + b
    print(f"result: {result}")
    
    return result

## Sequential implementation of GPT-2

In [23]:
logits = gpt2(inputs, **params, n_head=1)  # model forward pass

NameError: name 'softmax' is not defined

In [ ]:
logits

In [7]:
logits1 = layer_norm(logits,g,b)  # [n_seq, n_embd] -> [n_seq, n_embd]

Layer Norm Input Shapes:
x: [[ 0.12479967 -0.18312944  0.12330008 ... -0.23028293  0.07496844
   0.00799115]
 [-0.07021496 -0.13564175 -0.01681434 ... -0.00921412 -0.06527534
  -0.11999762]
 [-0.01704012 -0.25538594  0.16463962 ... -0.04039633 -0.02198055
   0.02862293]
 [ 0.11016754  0.00768144  0.16934796 ... -0.16861892  0.14384717
  -0.0459881 ]
 [-0.19569312  0.02489605  0.33249626 ...  0.05738477 -0.05327084
   0.03864006]]
x shape: (5, 768)
g shape: (768,)
b shape: (768,)
mean: [[-0.00529166]
 [-0.00109631]
 [-0.00258127]
 [-0.00253499]
 [-0.00225833]]
variance: [[0.14235575]
 [0.05211692]
 [0.04362927]
 [0.03986736]
 [0.04037109]]
x_normalized: [[ 0.3447828  -0.4713259   0.3408084  ... -0.59629744  0.21271442
   0.03520361]
 [-0.30273613 -0.58930206 -0.06884418 ... -0.0355556  -0.28110084
  -0.5207816 ]
 [-0.06921417 -1.2101698   0.8004824  ... -0.18101977 -0.09286391
   0.14937373]
 [ 0.5643785   0.05116062  0.86073524 ... -0.8316957   0.7330354
  -0.21759939]
 [-0.9625994   0

In [9]:
logits1.shape

(5, 768)

## End